In [1]:
#import libraries

from rdflib import Graph, Literal, RDF, URIRef, Namespace 
from rdflib.namespace import FOAF , XSD, SSN, SOSA 
import urllib.parse
import pandas as pd 
import matplotlib.pylab as plt
from matplotlib import pyplot

In [2]:
#read in the csv file 

df = pd.read_csv('soilpro.csv')

In [3]:
df.tail(2)

,Identifier,Site,Land_Use,Plot_Name,Subplot,Horizon,Soil_Moisture,Horizon_Depth,Bulk_Density,Soil_pH,total_C,total_N,Inorganic_P,C:N,C:P
220,SOP93,SAFE Project - Kalabakan Forest Reserve,Oil Palm,OP,9,Organic,12.26,3.0,0.80,6.63,2.79,0.23,48.31,12.08,0.06
221,SOP94,SAFE Project - Kalabakan Forest Reserve,Oil Palm,OP,9,Organic,10.83,5.5,1.07,6.53,6.06,0.43,126.75,13.94,0.05


In [ ]:
#decalring prefixes 

g = Graph()
ID = Namespace('SoilPropertiesCeh_')
SOSA = Namespace('http://www.w3.org/ns/sosa/')
UNIT= Namespace('http://qudt.org/vocab/unit/')
schema = Namespace('http://schema.org/')
uri=URIRef('http://www.w3.org/2000/01/rdf-schema#')
OBSPRO= Namespace('http://www.w3.org/ns/sosa/ObservableProperty/')
FEATURE= Namespace('http://www.w3.org/ns/sosa/FeatureOfInterest/')
TIME = Namespace('http://www.w3.org/2006/time#')
VOID = Namespace('http://rdfs.org/ns/void#')
XMLNS = Namespace('http://www.w3.org/XML/1998/namespace')
DGFC = Namespace('http://www.DGFC/ontology/2021/1.0/')

In [ ]:
#serialising the graph

for index, row in df.iterrows():
    g.add((URIRef(ID+row['Identifier']), RDF.type, SOSA.Observation))
    
    g.add((URIRef(ID+row['Identifier']), SOSA.Observation, Literal(row['Identifier'], datatype=XSD.string)))
    
    g.add((URIRef(ID+row['Identifier']), URIRef('DGFC'), Literal(row['Identifier'], datatype=XSD.string)))
      
    g.add((URIRef(ID+row['Identifier']), FEATURE.Site, Literal(row['Site'], datatype=XSD.string)))
    
    g.add((URIRef(ID+row['Identifier']), DGFC.Land_Use, Literal(row['Land_Use'], datatype=XSD.string)))
    
    g.add((URIRef(ID+row['Identifier']), DGFC.Plot_Name, Literal(row['Plot_Name'], datatype=XSD.string)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.Subplot, Literal(row['Subplot'], datatype=XSD.integer))) 

    g.add((URIRef(ID+row['Identifier']), OBSPRO.Horizon, Literal(row['Horizon'], datatype=XSD.string))) 
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.Soil_Moisture, Literal(row['Soil_Moisture'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.Horizon_Depth, Literal(row['Horizon_Depth'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.Bulk_Density, Literal(row['Bulk_Density'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.Soil_pH, Literal(row['Soil_pH'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.total_C, Literal(row['total_C'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.total_N, Literal(row['total_N'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.CtoN, Literal(row['C:N'], datatype=XSD.float)))
    
    g.add((URIRef(ID+row['Identifier']), OBSPRO.CtoP, Literal(row['C:P'], datatype=XSD.float)))
    

In [ ]:
# print(g.serialize(format='turtle').decode('UTF-8'))

In [ ]:
# saving graph to disk
g.serialize('soil.rdf', format='ttl')

In [ ]:
# Code to add data to stardog 
import stardog

conn_details = {
  'endpoint': 'http://localhost:5820',
  'username': 'admin',
  'password': 'admin'
}
with stardog.Admin(**conn_details) as admin:
    soil = admin.new_database('soil')

conn = stardog.Connection('soil', **conn_details)

conn.begin()

conn.add(
    stardog.content.File('soil.rdf', stardog.content_types.TURTLE)
#     graph_uri=graph_uri)
conn.commit()